In [1]:
#Parte 1
#Instalar Java8, descagar spark 2.4.8 con Hadoop
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
# instalar pyspark\n
!pip install -q pyspark
!pip install -q findspark
#crea variables de entorno
import os # libreria de manejo del sistema operativo
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ["PYTHON_PATH"] = "/content/spark-2.4.8-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip"

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 69.4 MB/s 


In [2]:
#Crear el SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('Control1')\
        .getOrCreate()
#spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=Control1>

In [3]:
#Se cargan los datos en la carpeta "Datos"
#Posteriormente deberá leer los datos del archivo de países y convertirlos en un RDD.
src='./Datos/CA Sales.csv'
rddCA = sc.textFile(src)
print('Cantidad CA:',rddCA.count())

src='./Datos/DE Sales.csv'
rddDE = sc.textFile(src)
print('Cantidad DE:',rddDE.count())

src='./Datos/FR Sales.csv'
rddFR = sc.textFile(src)
print('Cantidad FR:',rddFR.count())

src='./Datos/MX Sales.csv'
rddMX = sc.textFile(src)
print('Cantidad MX:',rddMX.count())

#El RDD creado debe reunir el contenido de los archivos correspondiente a CA_sales.csv,
#FR_sales.csv, DE_sales.csv, MX_sales.csv y mostrar la cantidad de filas que contiene RDD.
src='./Datos/*.csv'
rddALL = sc.textFile(src)
print('Total general:', rddALL.count())


Cantidad CA: 75319
Cantidad DE: 234073
Cantidad FR: 314751
Cantidad MX: 217008
Total general: 841362


In [6]:
#Se pide realizar sobre el RDD el proceso map reduce que permita separar cada uno
#de los elementos del RDD y posteriormente contar la cantidad de veces que se encuentra cada
#uno de los elementos dentro del archivo.

#Reduce los elementos
elementosALL = rddALL.flatMap(lambda x: x.split(","))
elementosALL.collect()[0:17]

wc = elementosALL.map(lambda x: (x,1))
#wc.collect()[0:10]

counts = wc.reduceByKey(lambda x,y:x+y)
counts.collect()[0:10]

[('1/15/1999', 1724),
 ('1', 817736),
 ('839.95', 2202),
 ('251.95', 1053),
 ('2188', 104),
 ('293.95', 3911),
 ('2009', 156),
 ('2282', 255),
 ('304.45', 4807),
 ('2255', 5320)]

In [7]:
#Crear un nuevo RDD que filtre los 4 países que aparecen en el archivo y el número de
#veces que se encuentre ordenado de mayor a menor y guardarlo en una carpeta llamada
#TotalPaises

countsr = counts.map(lambda x: (x[1],x[0])).sortByKey(ascending=False)
print(countsr.collect())

rddFin=countsr #.filter(lambda x : x[0]> 70000 and x[0] < 400000)
rddFin.collect()[0:10]
rddFin.saveAsTextFile("./Datos/TotalPaises/Salida")

#Fin parte 1

[(817736, '1'), (314750, 'France'), (234072, 'Germany'), (217007, 'Mexico'), (75318, 'Canada'), (20216, '2'), (14414, '89.20'), (11915, '475'), (11309, '604'), (10995, '487'), (10309, '419.95'), (10298, '157.45'), (9677, '309.70'), (9230, '438'), (8926, '826'), (8614, '314.95'), (8536, '57.70'), (8203, '68.20'), (7543, '2332'), (7296, '792'), (7296, '791'), (7087, '491'), (7074, '104.95'), (7048, '506'), (6896, '396'), (6886, '2045'), (6751, '83.95'), (6714, '60.32'), (6536, '70.82'), (6441, '1182'), (6356, '330.70'), (6328, '367.45'), (6266, '524.95'), (6235, '2396'), (6103, '262.45'), (6080, '65.57'), (6076, '449'), (6041, '609'), (6023, '997.45'), (5856, '4/15/1999'), (5753, '99.70'), (5732, '1000'), (5692, '5/15/1999'), (5670, '1102.45'), (5491, '1009'), (5366, '115.45'), (5320, '2255'), (5320, '2254'), (5287, '125.95'), (5281, '120.70'), (5217, '923.74'), (5213, '325.45'), (5208, '136.45'), (5114, '2388'), (5067, '927'), (4992, '725'), (4992, '726'), (4989, '786'), (4989, '785'), 

In [28]:
#Parte 2
#pide crear un DataFrame que contenga los datos de los archivos empleado.csv,
#Cargo.csv y departamento.csv. Para realizar esto se pide utilizar la función join.
archivo = './Datos/Empleado.csv'
Empleado= spark.read.csv(archivo,sep=';', inferSchema=True, header=True)

archivo = './Datos/cargo.csv'
Cargo= spark.read.csv(archivo,sep=';', inferSchema=True, header=True)
#Se actualiza nombre de columna que representa el nombre del cargo
Cargo= Cargo.withColumnRenamed('Cargo','Nom_Cargo')

archivo = './Datos/departamento.csv'
Departamento= spark.read.csv(archivo,sep=';', inferSchema=True, header=True)

UnionEC = Empleado.join(Cargo,Empleado["Cargo"]== Cargo["Codigo"])

UnionAll=UnionEC.join(Departamento,UnionEC["Departamento"]== Departamento["Codigo"])
UnionAll.show()

+-----------+------------------+----------------+------------------+----------+------+------------+-----+------------------+------+-----------------+-------+------+------+
|ID Empleado|   Nombre Empleado|          Estado|              Pais|Nacimiento|Género|Departamento|Cargo|       Nombre Jefe|Codigo|        Nom_Cargo| Sueldo|Codigo|Nombre|
+-----------+------------------+----------------+------------------+----------+------+------------+-----+------------------+------+-----------------+-------+------+------+
| 1102024149|    Spirea, Kelley|   California   |    Estados Unidos|  28-09-80|Female|           1|    2|      Elijiah Gray|     2|Admiiswtrativo II| 600000|     1|  RRHH|
| 1001109612|   Darson, Jene'ya|   California   |    Estados Unidos|  10-05-83|Female|           1|    2|      Elijiah Gray|     2|Admiiswtrativo II| 600000|     1|  RRHH|
| 1000974650|    Stanley, David|        Texas   |    Estados Unidos|  16-12-75|  Male|           1|    6|    Debra Houlihan|     6|         

In [29]:
#Eliminar del DataFrame las columnas Id_Empleado, Nombre_Empleado, Nombre_jefe,
#país.
UnionAll=UnionAll.drop("ID Empleado","Nombre Empleado","Nombre Jefe","Pais")
UnionAll.show()

+----------------+----------+------+------------+-----+------+-----------------+-------+------+------+
|          Estado|Nacimiento|Género|Departamento|Cargo|Codigo|        Nom_Cargo| Sueldo|Codigo|Nombre|
+----------------+----------+------+------------+-----+------+-----------------+-------+------+------+
|   California   |  28-09-80|Female|           1|    2|     2|Admiiswtrativo II| 600000|     1|  RRHH|
|   California   |  10-05-83|Female|           1|    2|     2|Admiiswtrativo II| 600000|     1|  RRHH|
|        Texas   |  16-12-75|  Male|           1|    6|     6|          Manager|4000000|     1|  RRHH|
|Massachusetts   |  10-06-69|  Male|           1|    6|     6|          Manager|4000000|     1|  RRHH|
|   California   |  02-04-72|Female|           1|    6|     6|          Manager|4000000|     1|  RRHH|
|   California   |  26-08-77|Female|           1|    2|     2|Admiiswtrativo II| 600000|     1|  RRHH|
|   California   |  06-02-85|Female|           1|    2|     2|Admiiswtrat

In [30]:
#Determinar la cantidad de filas del DataFrame, junto al tipo de datos que contiene y su
#esquema.
print('Cantidad de datos:',UnionAll.count())
print('Tipos de datos:',UnionAll.dtypes)
print('Esquema:')
UnionAll.printSchema()

Cantidad de datos: 194
Tipos de datos: [('Estado', 'string'), ('Nacimiento', 'string'), ('Género', 'string'), ('Departamento', 'int'), ('Cargo', 'int'), ('Codigo', 'int'), ('Nom_Cargo', 'string'), ('Sueldo', 'int'), ('Codigo', 'int'), ('Nombre', 'string')]
Esquema:
root
 |-- Estado: string (nullable = true)
 |-- Nacimiento: string (nullable = true)
 |-- Género: string (nullable = true)
 |-- Departamento: integer (nullable = true)
 |-- Cargo: integer (nullable = true)
 |-- Codigo: integer (nullable = true)
 |-- Nom_Cargo: string (nullable = true)
 |-- Sueldo: integer (nullable = true)
 |-- Codigo: integer (nullable = true)
 |-- Nombre: string (nullable = true)



In [31]:
#Carga de librerias para SQL
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [32]:
#Crear un nuevo DataFrame llamado Estado que permita obtener suma, promedio,
#máximo, mínimo de sueldo de los empleados por agrupado por estado.
Estado=(UnionAll.groupBy("Estado").agg(
      sum("Sueldo").alias("Suma_Sueldos"),
      round(avg("Sueldo"),1).alias("Promedio_sueldos"), 
      max("Sueldo").alias("Maximo_Sueldo"),
      min("Sueldo").alias("Minino_Sueldo")))
Estado.show()

+----------------+------------+----------------+-------------+-------------+
|          Estado|Suma_Sueldos|Promedio_sueldos|Maximo_Sueldo|Minino_Sueldo|
+----------------+------------+----------------+-------------+-------------+
|       Oregon   |     2500000|       2500000.0|      2500000|      2500000|
|      Indiana   |     2000000|       2000000.0|      2000000|      2000000|
|        Texas   |   134800000|       1750649.4|      5000000|       500000|
|    Tennessee   |     2500000|       2500000.0|      2500000|      2500000|
|     Colorado   |     2000000|       2000000.0|      2000000|      2000000|
| North Dakota   |     2000000|       2000000.0|      2000000|      2000000|
|Massachusetts   |   119300000|       2339215.7|      5000000|       500000|
|   Nueva York   |     2500000|       2500000.0|      2500000|      2500000|
|   California   |   106300000|       2005660.4|      5000000|       500000|
|       Nevada   |     2500000|       2500000.0|      2500000|      2500000|

In [33]:
#Crear un nuevo DataFrame llamado Departamento que permita obtener suma,
#promedio, máximo, mínimo de sueldo de los empleados por agrupado por departamento.
Departamento=(UnionAll.groupBy("Departamento").agg(
      sum("Sueldo").alias("Suma_Sueldos"),
      round(avg("Sueldo"),1).alias("Promedio_sueldos"), 
      max("Sueldo").alias("Maximo_Sueldo"),
      min("Sueldo").alias("Minino_Sueldo")))
Departamento.show()

+------------+------------+----------------+-------------+-------------+
|Departamento|Suma_Sueldos|Promedio_sueldos|Maximo_Sueldo|Minino_Sueldo|
+------------+------------+----------------+-------------+-------------+
|           1|    39900000|       1140000.0|      4000000|       500000|
|           6|    45500000|       2394736.8|      4000000|      2000000|
|           3|    95500000|       2387500.0|      5000000|      2000000|
|           5|    54500000|       2725000.0|      5000000|      2000000|
|           4|   120500000|       1650684.9|      5000000|      1000000|
|           2|    35000000|       5000000.0|      5000000|      5000000|
+------------+------------+----------------+-------------+-------------+



In [34]:
#Crear un nuevo DataFrame llamado Sexo que permita obtener suma, promedio,
#máximo, mínimo de sueldo de los empleados por agrupado por sexo
Sexo=(UnionAll.groupBy("Género").agg(
      sum("Sueldo").alias("Suma_Sueldos"),
      round(avg("Sueldo"),1).alias("Promedio_sueldos"), 
      max("Sueldo").alias("Maximo_Sueldo"),
      min("Sueldo").alias("Minino_Sueldo")))
Sexo.show()

+------+------------+----------------+-------------+-------------+
|Género|Suma_Sueldos|Promedio_sueldos|Maximo_Sueldo|Minino_Sueldo|
+------+------------+----------------+-------------+-------------+
|Female|   200400000|       1855555.6|      5000000|       500000|
|  Male|   190500000|       2215116.3|      5000000|       500000|
+------+------------+----------------+-------------+-------------+



In [35]:
#Crear un nuevo DataFrame llamado cargo que permita obtener suma, promedio,
#máximo, mínimo de sueldo de los empleados por agrupado por cargo.
cargo=(UnionAll.groupBy("Nom_Cargo").agg(
      sum("Sueldo").alias("Suma_Sueldos"),
      round(avg("Sueldo"),1).alias("Promedio_sueldos"), 
      max("Sueldo").alias("Maximo_Sueldo"),
      min("Sueldo").alias("Minino_Sueldo")))
cargo.show()

+-----------------+------------+----------------+-------------+-------------+
|        Nom_Cargo|Suma_Sueldos|Promedio_sueldos|Maximo_Sueldo|Minino_Sueldo|
+-----------------+------------+----------------+-------------+-------------+
|         Director|    60000000|       5000000.0|      5000000|      5000000|
| Administrativo I|     7500000|        500000.0|       500000|       500000|
|        Tecnico I|    30000000|       1000000.0|      1000000|      1000000|
|      Ingeniero I|    78000000|       2000000.0|      2000000|      2000000|
|Admiiswtrativo II|     8400000|        600000.0|       600000|       600000|
|       Tecnico II|    49500000|       1500000.0|      1500000|      1500000|
|          Manager|    80000000|       4000000.0|      4000000|      4000000|
|     Ingeniero II|    77500000|       2500000.0|      2500000|      2500000|
+-----------------+------------+----------------+-------------+-------------+



In [37]:
#Guardar en archivos CSVs con el mismo nombre de los DataFrames (Estado,
#Departamento, Sexo, cargo).
Estado.write.csv("./Estados", sep=',', header=True, mode="overwrite")
Departamento.write.csv("./Departamentos", sep=',', header=True, mode="overwrite")
Sexo.write.csv("./Sexo", sep=',', header=True, mode="overwrite")
Cargo.write.csv("./Cargo", sep=',', header=True, mode="overwrite")
